# Esercizio 6

Dato un file in formato GTF (Gene Transfer Format) che annota un set di geni sulla stessa genomica di riferimento, e il file della genomica di riferimento (*genomic reference*) in formato FASTA, produrre in output:

- per ognuno degli esoni annotati:
    - l'elenco degli identificatori dei trascritti che includono un dato esone insieme al gene di riferimento
    - l'elenco degli identificatori dei trascritti (insieme al gene di riferimento) per cui un dato esone è coperto completamente da CDS, specificando (per ogni trascritto) la porzione dell'esone coperta da CDS e la sua suddivisione in codoni

***

Parametri in input:

- nome del file in formato GTF
- nome del file della *genomic reference* in formato FASTA

***

Requisiti:

- deve essere definita una funzione `reverse_complement_in_case()` che prenda come argomento una sequenza nucleotidica e un valore di strand e ne restituisca il reverse&complement se lo strand è `-`, altrimenti restituisce la sequenza così com'è
- deve essere definita una funzione `codon_separating()` che prenda come argomento una sequenza nucleotidica e il frame in `{0, 1, 2}` e ne restituisca la suddivisione in codoni


**NOTA BENE**: gli attributi (coppie *nome-valore*) del nono campo del file GTF non devono essere pensati a ordine fisso all'interno del campo. Per estrarre quindi un attributo, non si può usare il metodo `split()`, ma si deve necessariamente usare un'espressione regolare.

***

Variabili di output:
- `exon_inclusion_dict`: dizionario di inclusione degli esoni: la chiave è la tupla *(start, end)* dell'esone e il valore è il set delle tuple *(transcript_id, gene_id)* che includono l'esone
- `exon_coverage_list`: lista degli esoni coperti interamente da coding sequence con la relativa suddivisione in codoni; ogni elemento è una tuple di cinque elementi: identificatore del trascritto, identificatore del gene, start dell'esone, end dell'esone, sudivisione in codoni (tenendo conto del valore di frame).   

***

### Note sul formato GTF

#### Esone incluso in trascritti diversi

Se un dato esone, inteso come intervallo `[start, end]` di posizioni sulla genomica di riferimento, è incluso in n trascritti di un gene, allora nel file GTF ci saranno n *record* di tipo `exon` con quei valori di start ed end. Gli identificatori di ognuno degli n trascritti che includono l'esone si trovano nel nono campo degli n *record*.

Ad esempio i *record* seguenti:

    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-014"; gene_id "ARHGAP4";
    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-002"; gene_id "ARHGAP4";
    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-003"; gene_id "ARHGAP4";
    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-001"; gene_id "ARHGAP4";
    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-024"; gene_id "ARHGAP4";
    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-011"; gene_id "ARHGAP4";
    
indicano che l'esone `[64566, 64757]` è incluso in sei trascritti del gene `ARHGAP4`.

#### Esone coperto da coding sequence

Se un dato esone `[start, end]`, incluso in un dato trascritto, è coperto (anche parzialmente) da CDS, allora ci sarà un *record* di tipo `CDS`, il cui nono campo contiene l'identificatore di tale trascritto, e il cui intervallo `[start_cds, end_cds]`, di inizio e fine sulla genomica di riferimento, coincide con l'intervallo `[start, end]` dell'esone, oppure ne occupa la parte finale (suffisso) o la parte iniziale (prefisso).

Ad esempio i due *record*:

    ENm006 VEGA_Known exon	70312	70440	.	-	.	transcript_id "U52112.4-005"; gene_id "ARHGAP4";
    ENm006 VEGA_Known CDS	70312	70440	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";

indicano che l'esone `[70312, 70440]` (incluso nel trascritto `U52112.4-005`del gene `ARHGAP4`) è coperto completamente dalla coding sequence del trascritto.

#### Suddivisione in codoni di una *feature* di tipo `CDS`

Dato un *record* di tipo `CDS`, la suddivisione in codoni della sequenza della *feature* sulla genomica di riferimento che esso rappresenta (attenzione: è una porzione della coding sequence del trascritto), deve tenere conto del valore del campo *frame* che specifica la posizione della prima base della sequenza all'interno del codone di appartenenza.

Ad esempio i tre *record* di tipo `CDS` seguenti:

    ENm006 VEGA_Known CDS	70312	70440	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";
    ENm006 VEGA_Known CDS	72521	72683	.	-	1	transcript_id "U52112.4-019"; gene_id "ARHGAP4";
    ENm006 VEGA_Known CDS	72761	72965	.	-	2	transcript_id "U52112.4-003"; gene_id "ARHGAP4";

rappresentano tre *features* di tipo `CDS` con i tre valori possibili del campo *frame*. 

Il valore 0 indica che la prima base della sequenza della *feature* CDS che esso rappresenta è la prima base di un codone (cioé le prime tre basi sono un codone completo).
Il valore 1 indica che la prima base è la seconda base di un codone, mentre 2 che la prima base è la terza base di un codone.

Le sequenze delle *features* rappresentate dai tre *record* sono rispettivamente:                        
 
    cggcaggccaagttcatggagcacaaactcaagtgcacaaaggcgcgcaacgagtacctgcttagcctggctagtgtcaacgctgctgtcagtaactactacctgcatgacgtcttggacctcatggac
    
    gaaggagccgtccctcctgtcgcccttgcactgctgggcggtgctgctgcagcacacgcggcagcagagccgggagagcgcggccctgagtgaggtgctggccgggcccctggcccagcgcctgagtcacattgcagaggacgtggggcgcctggtcaagaag
    
    agatgcgctggcagctgagcgagcagctgcgctgcctggagctgcagggcgagctgcggcgggagttgctgcaggagctggcagagttcatgcggcgccgcgctgaggtggagctggaatactcccggggcctggaaaagctggccgagcgcttctccagccgtggaggccgcctggggagcagccgggagcaccaaagcttccg

e le loro suddivisioni in codoni saranno dunque:

    cgg cag gcc aag ttc atg gag cac aaa ctc aag tgc aca aag gcg cgc aac gag tac ctg ctt agc ctg gct agt gtc aac gct gct gtc agt aac tac tac ctg cat gac gtc ttg gac ctc atg gac
    
    ga agg agc cgt ccc tcc tgt cgc cct tgc act gct ggg cgg tgc tgc tgc agc aca cgc ggc agc aga gcc ggg aga gcg cgg ccc tga gtg agg tgc tgg ccg ggc ccc tgg ccc agc gcc tga gtc aca ttg cag agg acg tgg ggc gcc tgg tca aga ag
    
    a gat gcg ctg gca gct gag cga gca gct gcg ctg cct gga gct gca ggg cga gct gcg gcg gga gtt gct gca gga gct ggc aga gtt cat gcg gcg ccg cgc tga ggt gga gct gga ata ctc ccg ggg cct gga aaa gct ggc cga gcg ctt ctc cag ccg tgg agg ccg cct ggg gag cag ccg gga gca cca aag ctt ccg

***

## Soluzione

Importare il modulo `re` per usare le espressioni regolari.

In [1]:
import re

### Definizione della funzione `reverse_complement_in_case()`

**NOTA BENE**: fare in modo che la funzione restituisca sempre una versione della sequenza in minuscolo.

### Parametri in input

In [4]:
gtf_file_name = './input.gtf'
reference_file_name = './ENm006.fa'

### Lettura della genomica di riferimento e memorizzazione nella variabile `genomic_reference`

Estrazione delle righe del file della genomica di riferimento nella lista `reference_file_rows`

In [6]:
reference_file_rows

['>ENm006\n',
 '...\n',
 ...]

Concatenazione delle righe contenenti la sequenza nucleotidica (dopo avere eliminato il simbolo di *newline* finale) nella variabile `genomic_reference`

In [8]:
genomic_reference

'...'

### Lettura dei *record* del file GTF e memorizzazione nella lista `gtf_file_rows`

In [10]:
gtf_file_rows

['ENm006\tVEGA_Known\texon\t71783\t71788\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t71783\t71788\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t70312\t70440\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t70312\t70440\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t69989\t70210\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t69989\t70210\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t64935\t65036\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64935\t65036\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t64566\t64673\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64566\t64673\t.\t-\t0\ttranscript_id "U52

### Selezione dei *record* di tipo `exon` e di tipo `CDS`

Separare i *record* di tipo `exon` e i *record* di tipo `CDS` in due liste distinte `exon_gtf_rows` e `cds_gtf_rows`

In [12]:
exon_gtf_rows

['ENm006\tVEGA_Known\texon\t71783\t71788\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t70312\t70440\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t69989\t70210\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t64935\t65036\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t64566\t64673\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t64385\t64459\t.\t-\t.\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t79484\t79511\t.\t-\t.\ttranscript_id "U52112.4-018"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t72761\t72965\t.\t-\t.\ttranscript_id "U52112.4-018"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t72521\t72683\t.\t-\t.\ttranscript_id "U52112.4-018"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\texon\t72253\t72379\t.\t-\t.\ttranscript_id

In [13]:
cds_gtf_rows

['ENm006\tVEGA_Known\tCDS\t71783\t71788\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t70312\t70440\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t69989\t70210\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64935\t65036\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64566\t64673\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t64385\t64459\t.\t-\t0\ttranscript_id "U52112.4-005"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t72761\t72963\t.\t-\t0\ttranscript_id "U52112.4-019"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t72521\t72683\t.\t-\t1\ttranscript_id "U52112.4-019"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t72253\t72315\t.\t-\t0\ttranscript_id "U52112.4-019"; gene_id "ARHGAP4";\n',
 'ENm006\tVEGA_Known\tCDS\t71872\t71965\t.\t-\t0\ttranscript_id "U52112.4

### Costruzione del dizionario `exon_inclusion_dict` di inclusione degli esoni

A partire dalla lista `exon_gtf_rows` costruire il dizionario `exon_inclusion_dict`:
- *chiave*: esone inteso come tupla *(start,end)*
- *valore*: set degli identificatori dei trascritti che includono l'esone (ogni elemento del set è una tupla *(transcript_id, gene_id)*)

e il dizionario `strand_dict`:
- *chiave*: identificatore del gene
- *valore*: strand 
    
Inizializzazione dei dizionari vuoti.

Attraversare la lista `exon_gtf_rows` per riempire i dizionari.

In [16]:
strand_dict

{'ARHGAP4': '-', 'ATP6AP1': '+', 'AVPR2': '+'}

In [17]:
exon_inclusion_dict

{(71783, 71788): {('U52112.4-005', 'ARHGAP4')},
 (70312, 70440): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-002', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-004', 'ARHGAP4'),
  ('U52112.4-005', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4'),
  ('U52112.4-014', 'ARHGAP4'),
  ('U52112.4-015', 'ARHGAP4'),
  ('U52112.4-020', 'ARHGAP4'),
  ('U52112.4-022', 'ARHGAP4')},
 (69989, 70210): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-002', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-005', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4'),
  ('U52112.4-012', 'ARHGAP4'),
  ('U52112.4-014', 'ARHGAP4'),
  ('U52112.4-022', 'ARHGAP4')},
 (64935, 65036): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-002', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-005', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4'),
  ('U52112.4-012', 'ARHGAP4'),
  ('U52112.4-013', 'ARHGAP4'),
  ('U52112.4-014', 'ARHGAP4'),
  ('U52112.4-022', 'ARHGAP4'),
  ('U52112.4-024', 'ARHGAP4')},
 (64566, 64673): {('U52112.4-005'

### Determinazione delle coperture degli esoni in termini di coding sequence

A partire dalla lista `cds_gtf_rows` costruire il dizionario `cds_inclusion_dict`:
- *chiave*: *feature* di tipo `CDS` intesa come tupla *(start,end)*
- *valore*: set degli identificatori dei trascritti che includono tale *feature* (ogni elemento del set è una tupla *(transcript_id, gene_id)*)

Inizializzazione del dizionario vuoto.

Attraversare la lista `exon_gtf_rows` per riempire il dizionario `cds_inclusion_dict`.

In [20]:
cds_inclusion_dict

{(71783, 71788): {('U52112.4-005', 'ARHGAP4')},
 (70312, 70440): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-005', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4'),
  ('U52112.4-020', 'ARHGAP4')},
 (69989, 70210): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-005', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4')},
 (64935, 65036): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-005', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4'),
  ('U52112.4-024', 'ARHGAP4')},
 (64566, 64673): {('U52112.4-005', 'ARHGAP4')},
 (64385, 64459): {('U52112.4-005', 'ARHGAP4')},
 (72761, 72963): {('U52112.4-017', 'ARHGAP4'), ('U52112.4-019', 'ARHGAP4')},
 (72521, 72683): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4'),
  ('U52112.4-017', 'ARHGAP4'),
  ('U52112.4-019', 'ARHGAP4'),
  ('U52112.4-024', 'ARHGAP4')},
 (72253, 72315): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-

A partire dalla lista `cds_gtf_rows` costruire il dizionario `frame_dict`:
- *chiave*: *feature* di tipo `CDS` intesa come tupla *(start,end)*
- *valore*: dizionario annidato:
    - *chiave*: identificatore del gene
    - *valore*: dizionario annidato:
        - *chiave*: identificatore del trascritto
        - *valore*: frame
    
Inizializzazione del dizionario vuoto.

Attraversare la lista `exon_gtf_rows` per riempire il dizionario `frame_dict`.

In [23]:
frame_dict

{(71783, 71788): {'ARHGAP4': {'U52112.4-005': 0}},
 (70312,
  70440): {'ARHGAP4': {'U52112.4-005': 0,
   'U52112.4-020': 0,
   'U52112.4-003': 0,
   'U52112.4-001': 0,
   'U52112.4-011': 0}},
 (69989,
  70210): {'ARHGAP4': {'U52112.4-005': 0,
   'U52112.4-003': 0,
   'U52112.4-001': 0,
   'U52112.4-011': 0}},
 (64935,
  65036): {'ARHGAP4': {'U52112.4-005': 0,
   'U52112.4-003': 0,
   'U52112.4-001': 0,
   'U52112.4-024': 0,
   'U52112.4-011': 0}},
 (64566, 64673): {'ARHGAP4': {'U52112.4-005': 0}},
 (64385, 64459): {'ARHGAP4': {'U52112.4-005': 0}},
 (72761, 72963): {'ARHGAP4': {'U52112.4-019': 0, 'U52112.4-017': 0}},
 (72521,
  72683): {'ARHGAP4': {'U52112.4-019': 1,
   'U52112.4-017': 1,
   'U52112.4-003': 1,
   'U52112.4-001': 1,
   'U52112.4-024': 1,
   'U52112.4-011': 1}},
 (72253,
  72315): {'ARHGAP4': {'U52112.4-019': 0,
   'U52112.4-017': 0,
   'U52112.4-003': 0,
   'U52112.4-001': 0,
   'U52112.4-024': 0}},
 (71872, 71965): {'ARHGAP4': {'U52112.4-019': 0}},
 (71865, 71965): {'AR

Cancellare in `cds_inclusion_dict` le chiavi che non compaiono in `exon_inclusion_dict` (cioé le *features* di tipo `CDS` che non coprono completamente un esone).

Cancellare da ognuno dei set contenuti in `cds_inclusion_dict`, le tuple che non appartengono al set di `exon_inclusion_dict` corrispondente alla stessa chiave.

In [26]:
cds_inclusion_dict

{(71783, 71788): {('U52112.4-005', 'ARHGAP4')},
 (70312, 70440): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-005', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4'),
  ('U52112.4-020', 'ARHGAP4')},
 (69989, 70210): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-005', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4')},
 (64935, 65036): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-005', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4'),
  ('U52112.4-024', 'ARHGAP4')},
 (64566, 64673): {('U52112.4-005', 'ARHGAP4')},
 (64385, 64459): {('U52112.4-005', 'ARHGAP4')},
 (72521, 72683): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-011', 'ARHGAP4'),
  ('U52112.4-017', 'ARHGAP4'),
  ('U52112.4-019', 'ARHGAP4'),
  ('U52112.4-024', 'ARHGAP4')},
 (72253, 72315): {('U52112.4-001', 'ARHGAP4'),
  ('U52112.4-003', 'ARHGAP4'),
  ('U52112.4-017', 'ARHGAP4'),
  ('U52112.4-019', 'ARHGAP4'),
  ('U52112.4-024', 'ARHGAP4'

Attraversamento del dizionario `cds_inclusion_dict` per costruire la lista `exon_coverage_list` delle suddivisioni in codoni delle *features* di tipo `CDS`. Ogni elemento è una tupla di cinque elementi: identificatore del trascritto, identificatore del gene, start dell'esone, end dell'esone, sudivisione in codoni (tenendo conto del valore di frame).

In [28]:
exon_coverage_list

[('U52112.4-005', 'ARHGAP4', 71783, 71788, 'gag aag'),
 ('U52112.4-005',
  'ARHGAP4',
  70312,
  70440,
  'cgg cag gcc aag ttc atg gag cac aaa ctc aag tgc aca aag gcg cgc aac gag tac ctg ctt agc ctg gct agt gtc aac gct gct gtc agt aac tac tac ctg cat gac gtc ttg gac ctc atg gac'),
 ('U52112.4-001',
  'ARHGAP4',
  70312,
  70440,
  'cgg cag gcc aag ttc atg gag cac aaa ctc aag tgc aca aag gcg cgc aac gag tac ctg ctt agc ctg gct agt gtc aac gct gct gtc agt aac tac tac ctg cat gac gtc ttg gac ctc atg gac'),
 ('U52112.4-011',
  'ARHGAP4',
  70312,
  70440,
  'cgg cag gcc aag ttc atg gag cac aaa ctc aag tgc aca aag gcg cgc aac gag tac ctg ctt agc ctg gct agt gtc aac gct gct gtc agt aac tac tac ctg cat gac gtc ttg gac ctc atg gac'),
 ('U52112.4-020',
  'ARHGAP4',
  70312,
  70440,
  'cgg cag gcc aag ttc atg gag cac aaa ctc aag tgc aca aag gcg cgc aac gag tac ctg ctt agc ctg gct agt gtc aac gct gct gtc agt aac tac tac ctg cat gac gtc ttg gac ctc atg gac'),
 ('U52112.4-003',
  'ARHGAP4',
  7031